# Phase 0: 環境構築と基礎理解

このノートブックでは、機械学習に必要な環境をセットアップし、基礎概念を理解します。

## 🎯 このPhaseで学ぶこと
- 機械学習の基本概念（教師あり学習、分類問題）
- 必要なライブラリの役割
- 開発環境のセットアップ

## 1. ライブラリのインポート確認

必要なライブラリがすべてインストールされているか確認します。

In [ ]:
# データ処理
import pandas as pd
import numpy as np

# 機械学習
from sklearn.linear_model import LogisticRegression
import lightgbm as lgb

# 可視化
import matplotlib.pyplot as plt
import seaborn as sns

# 日本語フォント設定（Windows）
plt.rcParams['font.sans-serif'] = ['MS Gothic']
plt.rcParams['axes.unicode_minus'] = False

print("✓ All libraries imported successfully!")
print(f"pandas version: {pd.__version__}")
print(f"numpy version: {np.__version__}")
print(f"lightgbm version: {lgb.__version__}")

## 2. 既存のcommon.jravanライブラリを試す

KeibaCICDプロジェクトには、すでにJRA-VANデータにアクセスするためのライブラリが用意されています。

In [ ]:
import sys
from pathlib import Path

# プロジェクトルートをパスに追加
project_root = Path.cwd().parent
sys.path.insert(0, str(project_root))

print(f"プロジェクトルート: {project_root}")

In [ ]:
from common.jravan import (
    get_horse_id_by_name,
    analyze_horse_training,
    get_horse_info
)

print("✓ common.jravanライブラリのインポート成功!")

### 実際に馬のデータを取得してみる

有名な競走馬の情報を取得してみましょう。

In [ ]:
# 馬名を指定（例：ドウデュース）
horse_name = "ドウデュース"

# 馬の基本情報を取得
horse_info = get_horse_info(horse_name)

if horse_info:
    print(f"馬名: {horse_info['name']}")
    print(f"性別: {horse_info['sex']}")
    print(f"年齢: {horse_info['age']}歳")
    print(f"所属: {horse_info['tozai']}")
    print(f"調教師: {horse_info['trainer_name']}")
else:
    print(f"馬 '{horse_name}' が見つかりませんでした。")
    print("別の馬名で試してみてください。")

In [ ]:
# 調教データも取得してみる
race_date = "20260125"  # 対象レース日

training = analyze_horse_training(horse_name, race_date)

if "error" not in training:
    print(f"\n=== {horse_name} の調教データ ===")
    print(f"調教本数: {training['total_count']}本 ({training['count_label']})")
    print(f"タイム分類: {training['time_class']}")
    
    if training.get("final"):
        final = training["final"]
        print(f"\n最終追切: {final['date']} {final['center']}{final['location']}")
        print(f"4F={final['time_4f']:.1f}秒 [{final['speed_class']}]")
        print(f"終い1F={final['lap_1']:.1f}秒 [{final['lap_class']}]")
else:
    print(f"調教データが取得できませんでした: {training.get('error')}")

## 3. 機械学習の基本概念

### 教師あり学習とは？

```
[入力データ]     →     [モデル]     →     [予測結果]
馬の情報、調教                            着順予測
レース条件など                            (1着になるか？)

学習時: 過去のレース結果（答え）を使ってモデルを訓練
予測時: 未来のレースに対して予測を行う
```

### 2値分類問題

競馬予測を「馬券圏内に入るか？」という2値分類問題として扱います：
- **Positive (1)**: 馬券圏内（1-3着）
- **Negative (0)**: 馬券圏外（4着以下）

In [ ]:
# 簡単な例：ダミーデータで2値分類を体験
from sklearn.datasets import make_classification
from sklearn.model_selection import train_test_split

# ダミーデータ生成（特徴量4つ、2値分類）
X, y = make_classification(
    n_samples=100,      # サンプル数
    n_features=4,       # 特徴量の数
    n_informative=3,    # 有益な特徴量
    n_redundant=1,      # 冗長な特徴量
    random_state=42
)

# データを学習用とテスト用に分割
X_train, X_test, y_train, y_test = train_test_split(
    X, y, test_size=0.3, random_state=42
)

print(f"学習データ: {X_train.shape}")
print(f"テストデータ: {X_test.shape}")
print(f"\n目的変数の分布:")
print(pd.Series(y_train).value_counts())

In [ ]:
# ロジスティック回帰で学習
model = LogisticRegression()
model.fit(X_train, y_train)

print("✓ モデル学習完了!")

# 予測
y_pred = model.predict(X_test)

# 正解率
from sklearn.metrics import accuracy_score
accuracy = accuracy_score(y_test, y_pred)

print(f"正解率: {accuracy:.1%}")

## 4. データの可視化

機械学習では、データを可視化して理解することが重要です。

In [ ]:
# ダミーデータをDataFrameに変換
df_example = pd.DataFrame(
    X_train,
    columns=[f'特徴量{i+1}' for i in range(4)]
)
df_example['target'] = y_train

# 散布図で可視化
plt.figure(figsize=(10, 6))
colors = ['red' if t == 0 else 'blue' for t in y_train]
plt.scatter(df_example['特徴量1'], df_example['特徴量2'], c=colors, alpha=0.6)
plt.xlabel('特徴量1')
plt.ylabel('特徴量2')
plt.title('データの分布（赤=クラス0, 青=クラス1）')
plt.grid(True, alpha=0.3)
plt.show()

In [ ]:
# 相関行列を可視化
plt.figure(figsize=(8, 6))
sns.heatmap(df_example.corr(), annot=True, cmap='coolwarm', center=0)
plt.title('特徴量間の相関')
plt.show()

## ✅ Phase 0 チェックリスト

以下を確認してください：

- [ ] ライブラリがすべてインポートできた
- [ ] common.jravanから馬のデータが取得できた
- [ ] 機械学習の基本概念（教師あり学習、分類問題）を理解した
- [ ] 簡単な2値分類モデルを訓練できた
- [ ] データを可視化できた

すべて確認できたら、**Phase 1（データ理解と可視化）**に進みましょう！

次のノートブック: `01_data_exploration.ipynb`

## 📝 メモ欄

学習中に気づいたことや疑問をここに記録しましょう。

---

(ここに自由にメモを追記してください)

---